# Generating a hypernyms tree
This Python notebook provides a comprehensive guide on how to extract hypernym classes from the standard 1000 ImageNet classes and save the information related to these hypernyms. 

In [32]:
import json
import pandas as pd

data = open('../assets/imagenet/ILSVRC2012_classes_original.json')
data = data.read()
data = json.loads(data)

### Open file with image labels

In [88]:
!cp /Users/kirillbykov/Documents/Work/INVERT/data/imagenet_val/ILSVRC2012_val.csv ../assets/imagenet/ILSVRC2012_val.csv

In [100]:
df = pd.read_csv('/Users/kirillbykov/Documents/Work/INVERT/data/imagenet_val/ILSVRC2012_val.csv')
df

,image_name,n01440764,n01443537,n01484850,n01491361,n01494475,n01496331,n01498041,n01514668,n01514859,...,n12768682,n12985857,n12998815,n13037406,n13040303,n13044778,n13052670,n13054560,n13133613,n15075141
0,ILSVRC2012_val_00000001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ILSVRC2012_val_00000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ILSVRC2012_val_00000003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ILSVRC2012_val_00000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ILSVRC2012_val_00000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,ILSVRC2012_val_00049996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,ILSVRC2012_val_00049997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,ILSVRC2012_val_00049998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,ILSVRC2012_val_00049999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Collecting hypernyms

In [ ]:
! pip install nltk
! pip install wn
! python -m wn download omw-en:1.4  # the Open English WordNet 2021

In [101]:
import nltk
from nltk.corpus import wordnet as wn
hyper = lambda s: s.hypernyms()

hypernyms_df = pd.DataFrame()
for index, synset_id in data.items():
    synset = wn.synset_from_pos_and_offset('n',int(synset_id[1:]))
    for hypernym in list(synset.closure(hyper)):
        column_name = '%s%08d' % (hypernym.pos(), hypernym.offset())
        if column_name in hypernyms_df:
            #this is possible due to the fact that imagenet labels do not intersect
            hypernyms_df[column_name] += df[synset_id]
        else:
            hypernyms_df[column_name] = df[synset_id]

#(optional) delete all hypernyms that contain only one class (i.e. 50 images)
hypernyms_df = hypernyms_df.loc[:, (hypernyms_df.sum(axis=0) > 50)]

/Users/kirillbykov/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/kirillbykov/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/kirillbykov/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 9
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/kirillbykov/opt/anaconda3/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('animal.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/var/folders/jf/1rx81_w56yn3cp5dpjyj31480000gn/T/ipykernel_19701/3895467163.py:1

In [102]:
hypernyms_df

,n01439121,n01438208,n01428580,n02528163,n02514825,n02512053,n01473806,n01471682,n01466257,n00015388,...,n09433442,n09613191,n00007846,n13135832,n11669921,n11665372,n11552386,n13083586,n00017222,n12992868
0,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [103]:
df = pd.concat([df, hypernyms_df], axis=1, )
df

,image_name,n01440764,n01443537,n01484850,n01491361,n01494475,n01496331,n01498041,n01514668,n01514859,...,n09433442,n09613191,n00007846,n13135832,n11669921,n11665372,n11552386,n13083586,n00017222,n12992868
0,ILSVRC2012_val_00000001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ILSVRC2012_val_00000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ILSVRC2012_val_00000003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ILSVRC2012_val_00000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ILSVRC2012_val_00000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,ILSVRC2012_val_00049996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,ILSVRC2012_val_00049997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,ILSVRC2012_val_00049998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,ILSVRC2012_val_00049999,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [104]:
df.to_csv('../assets/imagenet/ILSVRC2012_val.csv', index = False)

### Create a JSON file with descriptions

In [105]:
from tqdm import tqdm

ILSVRC2012_label_description = {}

for i, label in tqdm(enumerate(df.columns[1:])):
    synset = wn.synset_from_pos_and_offset('n',int(label[1:]))
    name = synset.name()
    definition = synset.definition()
    ILSVRC2012_label_description[i] = {'name': name,
                                           'definition': definition,
                                           'offset': 'n' + label[1:]
                                          }

1473it [00:00, 283703.44it/s]


In [106]:
import json

with open('../assets/imagenet/ILSVRC2012_classes.json', 'w') as fp:
    json.dump(ILSVRC2012_label_description, fp)